In [2]:
import os
import pandas as pd

# Set the directory path
folder_path = r"C:\Users\acer\Desktop\Data_Prep_1\imagesTr"

# List all files in the directory
file_names = os.listdir(folder_path)

# Create a DataFrame with file names
df = pd.DataFrame(file_names, columns=['nii_file'])

# Display the DataFrame
print(df)

                      nii_file
0     4716872_2018_11_29_L.nii
1     4716872_2018_11_29_R.nii
2     4722136_2019_10_31_L.nii
3     4722136_2019_10_31_R.nii
4      474881_2014_09_25_L.nii
..                         ...
370  66700362_2023_07_03_R.nii
371  66700404_2023_08_26_L.nii
372  66700404_2023_08_26_R.nii
373  66700681_2023_11_16_L.nii
374  66700681_2023_11_16_R.nii

[375 rows x 1 columns]


In [3]:
import pandas as pd

# Specify the path to your Excel file
file_path = r"C:\Users\acer\Desktop\Data_Prep_1\Classification.xlsx"

# Read the Excel file
excel_df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
print(excel_df.head())


   Identifier      Note   Age Classification  C_Erosion  C_Subchondral cyst  \
0   47-4881 L       NaN  53.0           L-OA          1                   1   
1   47-4881 R       NaN  53.0           R-OA          1                   0   
2  54-21497 L  2014 6x6  26.0           L-OA          1                   0   
3  54-21497 R  2014 6x6  26.0      R-remodel          0                   0   
4  57-10397 L       NaN   NaN      L-remodel          0                   0   

   C_Generalized sclerosis  C_Osteophyte  C_Flattening  C_Cortical sclerosis  \
0                        0             1             1                     0   
1                        0             1             1                     0   
2                        0             0             0                     0   
3                        0             0             1                     0   
4                        0             0             1                     0   

   GF_Erosion  GF_Subchondral cyst  GF_Gener

In [4]:
Match_df = excel_df[["Identifier"]]
Match_df['nii_exist'] = df['nii_file']
Match_df['nii_exist'] = Match_df['nii_exist'].fillna(0)

C:\Users\acer\AppData\Local\Temp\ipykernel_10652\2291817854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Match_df['nii_exist'] = df['nii_file']
C:\Users\acer\AppData\Local\Temp\ipykernel_10652\2291817854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Match_df['nii_exist'] = Match_df['nii_exist'].fillna(0)


In [5]:
import pandas as pd

# Function to extract the pattern
def extract_pattern(value):

    if value == 0:
        return 0
    parts = value.split('_')
    
    if len(parts) >= 2:
        # Remove .nii if present
        parts[-1] = parts[-1].replace('.nii', '')
        return parts[0][:2] + '-' + parts[0][2:] + ' ' + parts[-1]
    return None

# Apply the function to Column1 and create a new column with the pattern
Match_df['Pattern'] = Match_df['nii_exist'].apply(extract_pattern)

# Check if the pattern exists in Column2
Match_df['Exists'] = Match_df['Pattern'].isin(Match_df["Identifier"])


C:\Users\acer\AppData\Local\Temp\ipykernel_10652\464573081.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Match_df['Pattern'] = Match_df['nii_exist'].apply(extract_pattern)
C:\Users\acer\AppData\Local\Temp\ipykernel_10652\464573081.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Match_df['Exists'] = Match_df['Pattern'].isin(Match_df["Identifier"])


In [6]:
Match_df

,Identifier,nii_exist,Pattern,Exists
0,47-4881 L,4716872_2018_11_29_L.nii,47-16872 L,True
1,47-4881 R,4716872_2018_11_29_R.nii,47-16872 R,True
2,54-21497 L,4722136_2019_10_31_L.nii,47-22136 L,True
3,54-21497 R,4722136_2019_10_31_R.nii,47-22136 R,True
4,57-10397 L,474881_2014_09_25_L.nii,47-4881 L,True
...,...,...,...,...
369,57-24983 L,66700362_2023_07_03_L.nii,66-700362 L,True
370,57-24983 R,66700362_2023_07_03_R.nii,66-700362 R,True
371,57-34770 L,66700404_2023_08_26_L.nii,66-700404 L,True
372,57-34770 R,66700404_2023_08_26_R.nii,66-700404 R,True


In [7]:
import pandas as pd

# Sample data

df1 = Match_df[['Pattern']]
df2 = Match_df[['Identifier']]

# Add helper columns for merging on counts
df1['orderPattern'] = df1.groupby('Pattern').cumcount()
df2['orderIdentifier'] = df2.groupby('Identifier').cumcount()

# Merge with indicator to track merge status
result = pd.merge(df1, df2, left_on=['Pattern', 'orderPattern'], right_on=['Identifier', 'orderIdentifier'], how='outer', indicator=True)


C:\Users\acer\AppData\Local\Temp\ipykernel_10652\2508222240.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['orderPattern'] = df1.groupby('Pattern').cumcount()
C:\Users\acer\AppData\Local\Temp\ipykernel_10652\2508222240.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['orderIdentifier'] = df2.groupby('Identifier').cumcount()


In [8]:
result

,Pattern,orderPattern,Identifier,orderIdentifier,_merge
0,47-16872 L,0.0,47-16872 L,0.0,both
1,47-16872 R,0.0,47-16872 R,0.0,both
2,47-22136 L,0.0,47-22136 L,0.0,both
3,47-22136 R,0.0,47-22136 R,0.0,both
4,47-4881 L,0.0,47-4881 L,0.0,both
...,...,...,...,...,...
374,66-700362 R,0.0,66-700362 R,0.0,both
375,66-700404 L,0.0,66-700404 L,0.0,both
376,66-700404 R,0.0,66-700404 R,0.0,both
377,66-700681 L,0.0,66-700681 L,0.0,both


In [9]:
# Entries in df1 without a match
unmatched_df1 = result.loc[result['_merge'] == 'left_only', ['Pattern', 'orderPattern']]

# Entries in df2 without a match
unmatched_df2 = result.loc[result['_merge'] == 'right_only', ['Identifier', 'orderIdentifier']]


In [10]:
# Clean up unmatched dataframes
unmatched_df1 = unmatched_df1.drop(['orderPattern'], axis=1).drop_duplicates()
unmatched_df2 = unmatched_df2.drop(['orderIdentifier'], axis=1).drop_duplicates()

In [11]:
print(".nii files without corresponding label :")
print(unmatched_df1)

.nii files without corresponding label :
        Pattern
154  60-38868 L
217  62-12734 L
310   64-8619 L
350   65-8582 L
357  66-20679 L


In [12]:
print("Label without corresponding .nii files:")
print(unmatched_df2)

Label without corresponding .nii files:
                Identifier
185             61-16015 R
261              62-7533 L
283              63-2829 R
356  66-20679 /66-700443 L
378            66-700681 R
